# Python for stock

In [1]:
import requests
import pandas as pd
import io
import re
import time
import datetime
import os

In [6]:
import os
path = os.getcwd()
path

'C:\\Users\\諭陞陳\\Desktop\\python'

## Crawler for stock data

* [臺灣證券交易所](https://www.twse.com.tw/zh/)

In [3]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def crawler(date_time):
    page_url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date_time +'&type=ALLBUT0999'
    page = requests.get(page_url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if text == '"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",':
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text[:-1] + '\n' for text in use_text[initial_point:]])))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"',''))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x: x.replace('=',''))
    return test_df

def trans_date(date_time):
    return ''.join(str(date_time).split(' ')[0].split('-'))

def parse_n_days(start_date,n):
    df_dict = {}
    now_date = start_date
    for i in range(n):
        time.sleep(5)
        now_date = now_date - datetime.timedelta(days=1)
        try:
            df = crawler(trans_date(now_date))
            print("Current date" + trans_date(now_date))
            df_dict.update({trans_date(now_date):df})
            print('Successful!!')
        except:
            print('Fails at' + str(now_date))
    return df_dict

In [9]:
data = parse_n_days(datetime.datetime.now(),5)

Fails at2019-06-29 16:37:26.315858
Current date20190628
Successful!!
Current date20190627
Successful!!
Current date20190626
Successful!!
Current date20190625
Successful!!


In [10]:
# creaye a folder at the first time
createFolder(path+'\stock\\')

In [11]:
for i in data.keys():
    #createFolder(r'C:\Users\諭陞陳\Desktop\python\stock\\'+i+'\\')
    data[i].to_csv(path+'\stock\\'+i+'.csv',encoding='utf-8')

## Create a Database

In [12]:
import pandas as pd
import sqlite3
import glob

In [13]:
All_csv_file = glob.glob(path+'\stock\\*.csv')
All_csv_file

['C:\\Users\\諭陞陳\\Desktop\\python\\stock\\20190625.csv',
 'C:\\Users\\諭陞陳\\Desktop\\python\\stock\\20190626.csv',
 'C:\\Users\\諭陞陳\\Desktop\\python\\stock\\20190627.csv',
 'C:\\Users\\諭陞陳\\Desktop\\python\\stock\\20190628.csv']

In [14]:
df = pd.read_csv(All_csv_file[0],encoding='utf-8', engine='python').iloc[:,1:] # set up engine parameter to solve Chinese working dir
df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"7,941,128","2,804","643,677,923",81.15,81.40,80.60,80.75,-,0.45,80.75,110,80.80,185,0.00
1,0051,元大中型100,"10,431",8,"332,134",32.06,32.06,31.80,31.80,-,0.07,31.70,2,31.82,1,0.00
2,0052,富邦科技,"104,000",7,"5,497,200",53.40,53.40,52.70,52.70,-,0.80,52.60,50,52.95,10,0.00
3,0053,元大電子,"25,000",8,"846,570",34.18,34.18,33.80,33.87,-,0.33,33.87,25,33.94,2,0.00
4,0054,元大台商50,"1,000",1,"22,020",22.02,22.02,22.02,22.02,-,0.02,21.93,1,22.02,3,0.00
5,0055,元大MSCI金融,"38,450",19,"697,810",18.15,18.15,18.13,18.13,-,0.01,18.12,50,18.14,2,0.00
6,0056,元大高股息,"4,695,867","2,493","124,524,188",26.60,26.62,26.46,26.47,-,0.13,26.46,200,26.47,57,0.00
7,0057,富邦摩台,"23,000",5,"1,172,600",51.20,51.20,50.95,50.95,-,0.40,50.90,50,51.10,60,0.00
8,0058,富邦發達,"23,000",5,"1,071,710",46.77,46.77,46.57,46.57,-,0.26,46.49,80,46.62,20,0.00
9,0059,富邦金融,"3,186",5,"147,434",46.28,46.29,46.26,46.26,-,0.08,46.29,20,46.38,20,0.00


In [15]:
%%time
dbname = 'TWStock.db'

#連接到我們的資料庫，如果沒有的話會重新建一個
db = sqlite3.connect(dbname)

for file_name in All_csv_file:
    pd.read_csv(file_name,encoding='utf-8', engine='python').iloc[:,1:].to_sql(file_name.replace('.csv',''),db,if_exists='replace')

Wall time: 980 ms


In [16]:
# 讀取sqlite檔案裡面的表格
dates_list = [file_name.replace('.csv','') for file_name in All_csv_file]
pd.read_sql(con=db,sql='SELECT * FROM' + ' "'+ dates_list[0] +'"').head()

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0,0050,元大台灣50,"7,941,128","2,804","643,677,923",81.15,81.40,80.60,80.75,-,0.45,80.75,110,80.80,185,0.00
1,1,0051,元大中型100,"10,431",8,"332,134",32.06,32.06,31.80,31.80,-,0.07,31.70,2,31.82,1,0.00
2,2,0052,富邦科技,"104,000",7,"5,497,200",53.40,53.40,52.70,52.70,-,0.80,52.60,50,52.95,10,0.00
3,3,0053,元大電子,"25,000",8,"846,570",34.18,34.18,33.80,33.87,-,0.33,33.87,25,33.94,2,0.00
4,4,0054,元大台商50,"1,000",1,"22,020",22.02,22.02,22.02,22.02,-,0.02,21.93,1,22.02,3,0.00


In [17]:
%%time
total_df = pd.DataFrame()
for date in dates_list:
    df = pd.read_sql(con=db,sql='SELECT * FROM' + ' "'+ date +'"')
    df['Date'] = date.split('\\')[-1]
    total_df = total_df.append(df)

Wall time: 43.9 ms


In [18]:
%%time
total_df.shape

Wall time: 0 ns


(4385, 18)

In [19]:
dbname_2 = 'TWStock_2'
db2 = sqlite3.connect(dbname_2)

In [20]:
%%time 
total_dict = dict(tuple(total_df.groupby('證券代號')))
for key in total_dict.keys():
    df = total_dict[key].iloc[:,2:]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by=['Date'])
    df.to_sql(key,db2,if_exists='replace')

Wall time: 42.6 s


In [21]:
%%time
#測試股票代號資料庫
print(pd.read_sql(con=db2,sql='SELECT * FROM "2330"').tail())
print(pd.read_sql(con=db2,sql='SELECT * FROM "1217"').tail())

   index 證券名稱        成交股數    成交筆數            成交金額     開盤價     最高價     最低價  \
0    409  台積電  29,735,283  11,130   7,118,002,390  241.00  241.50  237.00   
1    409  台積電  28,770,041  13,535   6,757,991,717  235.00  236.50  234.00   
2    409  台積電  42,007,834  14,983  10,063,332,494  236.00  241.50  236.00   
3    409  台積電  28,085,212   8,987   6,715,488,371  241.50  241.50  238.00   

      收盤價 漲跌(+/-)  漲跌價差  最後揭示買價 最後揭示買量  最後揭示賣價 最後揭示賣量    本益比  \
0  238.50       -   2.5  238.50    211  239.00  1,361  19.16   
1  234.50       -   4.0  234.50  1,537  235.00    180  18.84   
2  240.50       +   6.0  240.00    612  240.50    469  19.32   
3  239.00       -   1.5  238.50    380  239.00    641  19.20   

                  Date  
0  2019-06-25 00:00:00  
1  2019-06-26 00:00:00  
2  2019-06-27 00:00:00  
3  2019-06-28 00:00:00  
   index 證券名稱       成交股數   成交筆數        成交金額   開盤價   最高價   最低價   收盤價 漲跌(+/-)  \
0    151  愛之味    398,856    179   2,827,180  7.08  7.10  7.07  7.07       -   
1    151  